In [1]:
import io, os, sys, types
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell
def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path
    
    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path
        
class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path
    
    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)
        
        print ("importing Jupyter notebook from %s" % path)
                                       
        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)
        
        
        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod
        
        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__
        
        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod
class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}
    
    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return
        
        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)
        
        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

sys.meta_path.append(NotebookFinder())

In [1]:
import torch
from torch import nn

In [22]:
class MLP(nn.Module):
    # multi layer perceptron
    def __init__(self,**kwargs):
        super(MLP,self).__init__(**kwargs)
        self.hidden = nn.Linear(784,256)
        self.act = nn.ReLU()#激活函数
        self.output = nn.Linear(256,10)
            
    def forward(self,x):
        a = self.act(self.hidden(x))
        return self.output(a)

In [4]:
X = torch.rand(2,784)
net = MLP()

In [5]:
net

MLP(
  (hidden): Linear(in_features=784, out_features=256, bias=True)
  (act): ReLU()
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [6]:
net(X)

tensor([[-0.2105, -0.1074,  0.1726, -0.1078, -0.1291,  0.0884,  0.0003, -0.1515,
         -0.0669, -0.0409],
        [-0.0923,  0.0070,  0.1675, -0.0844, -0.0554,  0.0294, -0.0480, -0.1069,
          0.0136, -0.0077]], grad_fn=<AddmmBackward>)

In [7]:
type(net)

__main__.MLP

### Sequential

In [15]:
class MySequential(nn.Module):
    from collections import OrderedDict
    def __init__(self,*args):
        super(MySequential,self).__init__()
        if len(args)==1 and isinstance(args[0],OrderedDict):
            for key,module in args[0].items():
                self.add_module(key,module)
        else:
            for idx,module in enumerate(args):
                self.add_module(str(idx),module)
    
    def forward(self,inputs):
        for module in self._modules.values():
            inputs = module(inputs)
        return inputs

In [16]:
seq_net = MySequential(
    nn.Linear(784,256),
    nn.ReLU(),
    nn.Linear(256,10)
    )

In [17]:
seq_net

MySequential(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)

In [18]:
seq_net(X)

tensor([[ 0.0918,  0.0886, -0.2675,  0.1245,  0.1012, -0.0593, -0.2368,  0.0613,
         -0.0917,  0.0358],
        [ 0.0299,  0.2334, -0.2640,  0.2335,  0.1302, -0.1104, -0.1995, -0.0473,
         -0.0113, -0.0741]], grad_fn=<AddmmBackward>)

### ModuleList

modulelist仅仅是层的列表，并不能像Sequential那样进行顺序推理

In [19]:
list_net = nn.ModuleList([nn.Linear(784,256),nn.ReLU()])
list_net.append(nn.Linear(256,10))

ModuleList(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)

In [20]:
list_net[-1]

Linear(in_features=256, out_features=10, bias=True)

### ModuleDict

ModuleDict接收一个子模块的字典作为输入, 然后也可以类似字典那样进行添加访问操作

In [21]:
dict_net = nn.ModuleDict({
    'linear': nn.Linear(784, 256),
    'act': nn.ReLU(),
})
dict_net['output'] = nn.Linear(256, 10) # 添加
print(dict_net['linear']) # 访问
print(dict_net.output)
print(dict_net)
# net(torch.zeros(1, 784)) # 会报NotImplementedError


Linear(in_features=784, out_features=256, bias=True)
Linear(in_features=256, out_features=10, bias=True)
ModuleDict(
  (act): ReLU()
  (linear): Linear(in_features=784, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)


### 复杂的模型


In [27]:
class FancyMLP(nn.Module):
    def __init__(self,**kwargs):
        super(FancyMLP,self).__init__(**kwargs)
        self.rand_weight = torch.rand((20,20),requires_grad=False)#constant weight常数参数
        self.linear = nn.Linear(20,20)
    def forward(self,x):
        x=self.linear(x)
        x= nn.functional.relu(torch.mm(x,self.rand_weight.data)+1)
        
        x=self.linear(x)#复用全连接层
        while x.norm().item() > 1:
            x/=2
        if x.norm().item() <0.8:
            x*=10
        return x.sum()

In [29]:
X = torch.rand(2,20)

In [30]:
net = FancyMLP()
net

FancyMLP(
  (linear): Linear(in_features=20, out_features=20, bias=True)
)

In [31]:
net(X)

tensor(-0.6645, grad_fn=<SumBackward0>)